### Testing One Hot Encoding versus Integer Encoding when you have multi-level categorical variables

In [47]:
import feather
import pandas as pd
import numpy as np

In [48]:
train = feather.read_dataframe("/Users/alexpapiu/Documents/Data/Bimbo/val.feather")

In [149]:
train = pd.read_csv("/Users/alexpapiu/Documents/Data/Bimbo/train.csv")

In [49]:
train.shape

(1000000, 19)

In [50]:
train.head(3)

,Producto_ID,Agencia_ID,Semana,Canal_ID,Ruta_SAK,Cliente_ID,Demanda_uni_equil,id,Town,State,V1,product_name,brand,weight,pieces,weight_per_piece,has_choco,has_vanilla,has_multigrain
0,53,1374,9,4,6652,4578089,2,67467299,2251 AGUASCALIENTES NORTE,AGUASCALIENTES,4,Burritos Sincro 170g CU LON 53,LON,170,NaN,NaN,False,False,False
1,53,1374,9,4,6651,2155654,2,67466660,2251 AGUASCALIENTES NORTE,AGUASCALIENTES,4,Burritos Sincro 170g CU LON 53,LON,170,NaN,NaN,False,False,False
2,53,1374,9,4,6652,1011336,4,67467020,2251 AGUASCALIENTES NORTE,AGUASCALIENTES,4,Burritos Sincro 170g CU LON 53,LON,170,NaN,NaN,False,False,False


In [51]:
train.dtypes

Producto_ID            int32
Agencia_ID             int32
Semana                 int32
Canal_ID               int32
Ruta_SAK               int32
Cliente_ID             int32
Demanda_uni_equil      int32
id                     int32
Town                  object
State                 object
V1                    object
product_name          object
brand                 object
weight               float64
pieces               float64
weight_per_piece     float64
has_choco               bool
has_vanilla             bool
has_multigrain          bool
dtype: object

In [52]:
train.Agencia_ID.value_counts().index[10]

1312

In [157]:
train = train[train.Agencia_ID == 1945]

In [53]:
train.shape

(1000000, 19)

###  Now let's try to integer encode stuff and apply xgboost:

In [54]:
from sklearn import preprocessing

In [55]:
le = preprocessing.LabelEncoder()

In [62]:
X = np.column_stack((le.fit_transform(train.Producto_ID),
                     le.fit_transform(train.Canal_ID),
                     le.fit_transform(train.Cliente_ID),
                     le.fit_transform(train.Ruta_SAK),
                     le.fit_transform(train.Agencia_ID)))

In [60]:
y = np.log1p(train.Demanda_uni_equil)

In [61]:
X.shape

(1000000, 5)

In [67]:
import xgboost as xgb

In [165]:
dtrain = xgb.DMatrix(data = X, label = y)

In [172]:
params = {
        "eta": 0.7,
        "max_depth": 6,
        "silent": 1,
        "seed": 1233,
    }

In [174]:
xgb.cv(params, dtrain, num_boost_round=40)

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,0.668165,0.007096,0.667889,0.006556
1,0.557114,0.004477,0.556773,0.004630
2,0.541426,0.008108,0.540532,0.008582
3,0.529641,0.004832,0.528921,0.004814
4,0.518630,0.001407,0.517749,0.000564
5,0.514054,0.001735,0.513014,0.002480
6,0.511275,0.001290,0.510080,0.001945
7,0.508625,0.001980,0.507389,0.001903
8,0.506592,0.001481,0.505069,0.001090
9,0.503878,0.000887,0.502393,0.000258


### And let's also try One Hot Encoding:

In [175]:
le.fit

(570384, 11)

In [182]:
train.Cliente_ID = train.Cliente_ID.astype("str")
train.Producto_ID = train.Producto_ID.astype("str")


In [69]:
pd.get_dummies(train[1:1000][["Cliente_ID", "Producto_ID"]])

,Cliente_ID,Producto_ID
1,2155654,53
2,1011336,53
3,74428,72
4,416953,72
5,4661879,72
6,1393062,72
7,73880,72
8,1272481,72
9,4362105,72
10,2315372,72


In [37]:
X = np.array([[1, 2, 3, 3, 4], [1, 2, 3, 1, 1]])

In [43]:
X = np.column_stack(([1, 2, 3, 3, 4], [1, 2, 3, 1, 1]))

In [44]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(X)  

OneHotEncoder(categorical_features='all', dtype=<class 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [45]:
X.shape

(5, 2)

In [46]:
enc.transform(X).toarray()

array([[ 1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  0.]])